# iNaturalist status updates by state

Using the file produced in the collate-status-taxa.ipynb: `inat-aust-status-taxa.csv`, generate lists to update iNaturalist statuses

**Next steps:**
State by state establish the changes that need to be made:
    a. new - any new species that appear in the state lists but do not have a status in inaturalist (new template)
    b. updates - any changes to information which was added by us previously (user_id = 708886) (update template, action='UPDATE')
    c. removals - any statuses which were added by us previously (user_id = 708886) list which are incorrect (update template, action='REMOVE')
    d. flags - are there any statuses by other users that need to be flagged?

## Prep - common to all states
1. Read in the inaturalist statuses & filter out this state
2. Read in the inaturalist taxa list
3. Read in the state sensitive list
4. Attempt to match the state statuses to an IUCN equivalent


### 1. iNaturalist statuses

In [1]:
import sys
import os
projectdir = "/Users/new330/IdeaProjects/authoritative-lists/" # basedir for this gh project
#projectdir = "/Users/oco115/PycharmProjects/authoritative-lists/" # basedir for this gh project
sys.path.append(os.path.abspath(projectdir + "source-code/includes"))
import list_functions as lf

import pandas as pd
# projectdir = "/Users/new330/IdeaProjects/authoritative-lists/" # basedir for this gh project
sourcedir = projectdir + "source-data/inaturalist-statuses/"
listdir = projectdir + "current-lists/"


# read in the statuses
taxastatus = pd.read_csv(sourcedir + "inat-aust-status-taxa.csv", encoding='UTF-8',na_filter=False,dtype=str) ## Read inaturalist conservation statuses file
taxastatus.head(3)

,id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,...,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
0,166449,38493,1138587,7830,,Flora and Fauna Guarantee Act 1988,CR,,,obscured,...,Eulamprus,kosciuskoi,,2021-03-01T10:35:01Z,Eulamprus kosciuskoi,species,http://reptile-database.reptarium.cz/search.ph...,,,
1,234788,918383,702203,9994,,Atlas of Living Australia,NT,https://bie.ala.org.au/species/https://id.biod...,,,...,Chiloschista,phyllorhiza,,2022-01-08T03:30:36Z,Chiloschista phyllorhiza,species,http://www.catalogueoflife.org/annual-checklis...,,,
2,234789,918383,702203,7308,,Atlas of Living Australia,LC,https://bie.ala.org.au/species/https://id.biod...,,,...,Chiloschista,phyllorhiza,,2022-01-08T03:30:36Z,Chiloschista phyllorhiza,species,http://www.catalogueoflife.org/annual-checklis...,,,


In [2]:
def filter_state_statuses(stateregex: str, urlregex: str):
    authoritydf = taxastatus['authority'].drop_duplicates().sort_values()
    authoritydf = authoritydf[pd.Series(authoritydf).str.contains(stateregex)]
    urldf = taxastatus['url'].drop_duplicates().sort_values()
    urldf = urldf[pd.Series(urldf).str.contains(urlregex)]
    placedisplaydf = taxastatus['place_display_name'].drop_duplicates().sort_values()
    placedisplaydf = placedisplaydf[pd.Series(placedisplaydf).str.contains(stateregex)]
    placedf = taxastatus['place_name'].drop_duplicates().sort_values()
    placedf = placedf[pd.Series(placedf).str.contains(stateregex)]
    # concat all and remove duplicates
    statedf = pd.concat([taxastatus.apply(lambda row: row[taxastatus['place_display_name'].isin(placedisplaydf)]),
                         taxastatus.apply(lambda row: row[taxastatus['place_name'].isin(placedf)]),
                         taxastatus.apply(lambda row: row[taxastatus['url'].isin(urldf)]),
                         taxastatus.apply(
                             lambda row: row[taxastatus['authority'].isin(authoritydf)])]).drop_duplicates()
    return statedf.sort_values(['taxon_id', 'user_id'])
inatstatuses = filter_state_statuses("Tasmania|TAS", ".tas.gov")
inatstatuses.rename(columns={'id':'status_id','id_y':'taxon_id_y'},inplace=True)
inatstatuses

,status_id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,...,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
212,170159,1007040,702203,6829,,Threatened Species Protection Act 1995,EN,https://www.threatenedspecieslink.tas.gov.au/P...,,,...,Dianella,amoena,,2021-07-27T17:12:17Z,Dianella amoena,species,http://www.catalogueoflife.org/annual-checklis...,,,
2558,160921,1038747,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Ranunculus...,,obscured,...,Ranunculus,diminutus,,2022-06-13T17:38:59Z,Ranunculus diminutus,species,,,,
2547,160910,1038906,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Myosurus-a...,,obscured,...,Myosurus,australis,,2020-08-19T03:33:32Z,Myosurus australis,species,,,,
933,160738,1038964,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Boronia%20...,,obscured,...,Boronia,hemichiton,,2020-08-17T05:51:50Z,Boronia hemichiton,species,,,,
2557,160920,1038976,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Pimelea-ax...,,obscured,...,Pimelea,axiflora,axiflora,2020-12-02T04:00:26Z,Pimelea axiflora axiflora,subspecies,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2586,160759,958863,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Corunastyl...,,,...,Genoplesium,nudiscapum,,2022-10-26T09:34:12Z,Genoplesium nudiscapum,species,https://eol.org/pages/1099120,,,
2594,161103,960004,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Thelymitra...,,obscured,...,Thelymitra,bracteata,,2022-07-14T00:09:44Z,Thelymitra bracteata,species,http://www.catalogueoflife.org/annual-checklis...,,,
3380,268029,993886,1116633,6829,,TASMANIAN THREATENED SPECIES LISTING STATEMENT,EN,https://nre.tas.gov.au/Documents/Prasophyllum%...,1. total population estimated to number fewer ...,obscured,...,Prasophyllum,abblittiorum,,2020-02-18T23:45:10Z,Prasophyllum abblittiorum,species,https://eol.org/pages/51504614,,,
813,160748,993902,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Caladenia-...,,,...,Caladenia,dienema,,2020-12-15T00:50:01Z,Caladenia dienema,species,http://www.catalogueoflife.org/annual-checklis...,,,


### 2. iNaturalist taxonomy

In [3]:
# Output files contain these fields
# Taxon_Name,Status,Authority,IUCN_equivalent,Description,iNaturalist_Place_ID,url,Taxon_Geoprivacy,Username,taxon_id
# so we need to match species from the state lists to the inat taxa to get the taxon_id

import zipfile
url = "https://www.inaturalist.org/taxa/inaturalist-taxonomy.dwca.zip"
filename = url.split("/")[-1]

z=zipfile.ZipFile(sourcedir + filename)

with z.open('taxa.csv') as from_archive:
    inattaxa = pd.read_csv(from_archive,dtype=str)
z.close()
inattaxa.head(3)


,id,taxonID,identifier,parentNameUsageID,kingdom,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references
0,1,https://www.inaturalist.org/taxa/1,https://www.inaturalist.org/taxa/1,https://www.inaturalist.org/taxa/48460,Animalia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-02T06:05:44Z,Animalia,kingdom,http://www.catalogueoflife.org/annual-checklis...
1,2,https://www.inaturalist.org/taxa/2,https://www.inaturalist.org/taxa/2,https://www.inaturalist.org/taxa/1,Animalia,Chordata,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-23T00:40:18Z,Chordata,phylum,http://www.catalogueoflife.org/annual-checklis...
2,3,https://www.inaturalist.org/taxa/3,https://www.inaturalist.org/taxa/3,https://www.inaturalist.org/taxa/355675,Animalia,Chordata,Aves,NaN,NaN,NaN,NaN,NaN,2022-12-27T07:33:16Z,Aves,class,http://www.catalogueoflife.org/annual-checklis...


### 3. State lists
Sensitive list: `geoprivacy` = `obscured`
overrides

In [4]:
# %%script echo skipping # comment this line to download dataset from lists.ala.org.au the web and save locally
# Download lists data. Retrieve binomial and trinomial names from GBIF. Save locally to CSV

sensitivelist = lf.download_ala_list("https://lists.ala.org.au/ws/speciesListItems/dr491?max=10000&includeKVP=true")
# sensitivelist = lf.kvp_to_columns(sensitivelist)   # kvp values null
sensitivelist.to_csv(sourcedir + "tas-ala-sensitive.csv", index=False)

In [5]:
conservationlist = lf.download_ala_list("https://lists.ala.org.au/ws/speciesListItems/dr654?max=10000&includeKVP=true")
conservationlist = lf.kvp_to_columns(conservationlist)
conservationlist.to_csv(sourcedir + "tas-ala-conservation.csv", index=False)

In [19]:
# Read sensitive list data
sensitivelist = pd.read_csv(sourcedir + "tas-ala-sensitive.csv", dtype=str)
# Tasmania has only 1 sensitive species in list so set status to Conservation list status for the species
sensitivelist['status'] = 'Endangered'
sensitivelist['tas_geoprivacy'] = 'private'
sensitivelist

,id,name,commonName,scientificName,lsid,dataResourceUid,kvpValues,status,tas_geoprivacy
0,24386,Lomatia tasmanica,King's Lomatia,Lomatia tasmanica,https://id.biodiversity.org.au/node/apni/2903191,dr491,[],Endangered,private


In [8]:
# Read conservation list data
conservationlist = pd.read_csv(sourcedir + "tas-ala-conservation.csv", dtype=str)
conservationlist['tas_geoprivacy'] = 'open'
conservationlist['tas_geoprivacy'] = conservationlist['name'].apply(lambda x: 'private' if x == "Lomatia tasmanica" else 'open')
conservationlist

,id,name,commonName,scientificName,lsid,dataResourceUid,kvpValues,scientificNameAuthorship,vernacular name,family,class,sourceStatus,status,sourceKingdom,kingdom,speciesGroup,sourceOrigin,higherGeography,E P B C A_status,tas_geoprivacy
0,4033150,Costora iena,Caddis Fly (great Lakes),Costora iena,https://biodiversity.org.au/afd/taxa/7de808fb-...,dr654,"[{'key': 'scientificNameAuthorship', 'value': ...",Mosely,Caddis Fly (Great Lakes),Trichoptera,Invertebrates,x,Presumed Extinct,Fauna,Animalia,invertebrate,end,endemic Tasmania,NaN,open
1,4033128,Hadronyche pulvinator,Cascade Funnel-web Spider,Hadronyche pulvinator,https://biodiversity.org.au/afd/taxa/2acd0df1-...,dr654,"[{'key': 'scientificNameAuthorship', 'value': ...",Hickman,Cascade Funnel-web Spider,Araneae,Invertebrates,x,Presumed Extinct,Fauna,Animalia,invertebrate,end,endemic Tasmania,NaN,open
2,4033378,Hypolimnus pedderensis,Lake Pedder Earthworm,Hypolimnus pedderensis,https://biodiversity.org.au/afd/taxa/7db09aa3-...,dr654,"[{'key': 'scientificNameAuthorship', 'value': ...",(Jamieson),Lake Pedder Earthworm,Haplotaxida,Invertebrates,x,Presumed Extinct,Fauna,Animalia,invertebrate,end,endemic Tasmania,EX,open
3,4033234,Thylacinus cynocephalus,Thylacine,Thylacinus cynocephalus,https://biodiversity.org.au/afd/taxa/97764bed-...,dr654,"[{'key': 'scientificNameAuthorship', 'value': ...",Harris,Thylacine,Mammals,Vertebrates,x,Presumed Extinct,Fauna,Animalia,vertebrate,end,endemic Tasmania,EX,open
4,4033416,Gallirallus philippensis macquariensis,NaN,Gallirallus philippensis macquariensis,NZOR-6-101588,dr654,"[{'key': 'scientificNameAuthorship', 'value': ...",Hutton,Macquarie Island Rail,Birds,Vertebrates,x,Presumed Extinct,Fauna,Animalia,vertebrate,eMI,endemic Macquarie Island,EX,open
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678,4033369,Erioderma sorediatum,NaN,Erioderma sorediatum,ALA_DR654_679,dr654,"[{'key': 'scientificNameAuthorship', 'value': ...",D.J.Galloway & P.M.Jørg.,NaN,Pannariaceae,Lichen,e,Endangered,Flora,Plantae,non-vascular,NaN,NaN,NaN,open
679,4033419,Xanthoparmelia amphixantha,NaN,Xanthoparmelia amphixantha,ALA_DR654_680,dr654,"[{'key': 'scientificNameAuthorship', 'value': ...",(Müll.Arg.) Hale,NaN,Parmeliaceae,Lichen,e,Endangered,Flora,Plantae,non-vascular,NaN,NaN,NaN,open
680,4033291,Xanthoparmelia molliuscula,NaN,Xanthoparmelia molliuscula,ALA_DR654_681,dr654,"[{'key': 'scientificNameAuthorship', 'value': ...",(Ach.) Hale,NaN,Parmeliaceae,Lichen,e,Endangered,Flora,Plantae,non-vascular,NaN,NaN,NaN,open
681,4033236,Xanthoparmelia subloxodella,NaN,Xanthoparmelia subloxodella,ALA_DR654_682,dr654,"[{'key': 'scientificNameAuthorship', 'value': ...","(Elix & Kantvilas) O.Blanco, A.Crespo, Elix, D...",NaN,Parmeliaceae,Lichen,e,Endangered,Flora,Plantae,non-vascular,NaN,NaN,NaN,open


In [20]:
# join them in a way that works for inat (eg sensitive list, geoprivacy = 'obscured'
statelist = pd.concat([sensitivelist[['name','tas_geoprivacy', 'status', 'lsid']],
                    conservationlist[['name','tas_geoprivacy', 'status', 'lsid']]]).drop_duplicates()

In [11]:
# retrieve binomial and trinomial names from GBIF
parsednames = lf.gbifparse(statelist)
parsednames.to_csv(sourcedir + "tas-gbif.csv", index=False)

In [21]:
parsednames = pd.read_csv(sourcedir + "tas-gbif.csv")
statelist = statelist.merge(parsednames[['scientificName','canonicalName','canonicalNameComplete','type','rankMarker']],how="left",left_on="name",right_on="scientificName")
numfullstatelist = len(statelist.index)
statelist['scientificName'] = statelist['canonicalName']
statelist

,name,tas_geoprivacy,status,lsid,scientificName,canonicalName,canonicalNameComplete,type,rankMarker
0,Lomatia tasmanica,private,Endangered,https://id.biodiversity.org.au/node/apni/2903191,Lomatia tasmanica,Lomatia tasmanica,Lomatia tasmanica,SCIENTIFIC,sp.
1,Lomatia tasmanica,private,Endangered,https://id.biodiversity.org.au/node/apni/2903191,Lomatia tasmanica,Lomatia tasmanica,Lomatia tasmanica,SCIENTIFIC,sp.
2,Costora iena,open,Presumed Extinct,https://biodiversity.org.au/afd/taxa/7de808fb-...,Costora iena,Costora iena,Costora iena,SCIENTIFIC,sp.
3,Hadronyche pulvinator,open,Presumed Extinct,https://biodiversity.org.au/afd/taxa/2acd0df1-...,Hadronyche pulvinator,Hadronyche pulvinator,Hadronyche pulvinator,SCIENTIFIC,sp.
4,Hypolimnus pedderensis,open,Presumed Extinct,https://biodiversity.org.au/afd/taxa/7db09aa3-...,Hypolimnus pedderensis,Hypolimnus pedderensis,Hypolimnus pedderensis,SCIENTIFIC,sp.
...,...,...,...,...,...,...,...,...,...
679,Erioderma sorediatum,open,Endangered,ALA_DR654_679,Erioderma sorediatum,Erioderma sorediatum,Erioderma sorediatum,SCIENTIFIC,sp.
680,Xanthoparmelia amphixantha,open,Endangered,ALA_DR654_680,Xanthoparmelia amphixantha,Xanthoparmelia amphixantha,Xanthoparmelia amphixantha,SCIENTIFIC,sp.
681,Xanthoparmelia molliuscula,open,Endangered,ALA_DR654_681,Xanthoparmelia molliuscula,Xanthoparmelia molliuscula,Xanthoparmelia molliuscula,SCIENTIFIC,sp.
682,Xanthoparmelia subloxodella,open,Endangered,ALA_DR654_682,Xanthoparmelia subloxodella,Xanthoparmelia subloxodella,Xanthoparmelia subloxodella,SCIENTIFIC,sp.


In [22]:
# Identify records that won't comply with iNaturalist species names
noncomply = statelist[statelist['type'].isin(['INFORMAL','CULTIVAR','HYBRID']) ]
noncomply

,name,tas_geoprivacy,status,lsid,scientificName,canonicalName,canonicalNameComplete,type,rankMarker
79,"Charopidae ""Skemps""",open,Rare,ALA_DR654_78,Charopidae cv.,Charopidae cv.,Charopidae cv. 'Skemps',CULTIVAR,cv.
83,Enchymus sp. nov.,open,Rare,ALA_DR654_82,Enchymus spec.,Enchymus spec.,Enchymus spec.,INFORMAL,sp.
324,Isoetes sp. Maxwell River,open,Rare,ALA_DR654_323,Isoetes sp.Maxwell,Isoetes sp.Maxwell,Isoetes sp.Maxwell,INFORMAL,sp.
569,Euphrasia sp. Bivouac Bay,open,Endangered,ALA_DR654_568,Euphrasia sp.Bivouac,Euphrasia sp.Bivouac,Euphrasia sp.Bivouac,INFORMAL,sp.
587,Prasophyllum sp. Arthurs Lake,open,Endangered,ALA_DR654_587,Prasophyllum sp.Arthurs,Prasophyllum sp.Arthurs,Prasophyllum sp.Arthurs,INFORMAL,sp.
647,Pimelea sp. Tunbridge,open,Endangered,ALA_DR654_647,Pimelea sp.Tunbridge,Pimelea sp.Tunbridge,Pimelea sp.Tunbridge,INFORMAL,sp.
649,Polyscias sp. Douglas-Denison,open,Endangered,ALA_DR654_649,Polyscias sp.Douglas-Denison,Polyscias sp.Douglas-Denison,Polyscias sp.Douglas-Denison,INFORMAL,sp.


In [23]:
# remove records that do not comply
statelist = statelist[~statelist['type'].isin(['INFORMAL','CULTIVAR','HYBRID']) ]
statelist = pd.DataFrame(statelist[['scientificName','status','tas_geoprivacy','lsid']]).drop_duplicates()
statelist

,scientificName,status,tas_geoprivacy,lsid
0,Lomatia tasmanica,Endangered,private,https://id.biodiversity.org.au/node/apni/2903191
2,Costora iena,Presumed Extinct,open,https://biodiversity.org.au/afd/taxa/7de808fb-...
3,Hadronyche pulvinator,Presumed Extinct,open,https://biodiversity.org.au/afd/taxa/2acd0df1-...
4,Hypolimnus pedderensis,Presumed Extinct,open,https://biodiversity.org.au/afd/taxa/7db09aa3-...
5,Thylacinus cynocephalus,Presumed Extinct,open,https://biodiversity.org.au/afd/taxa/97764bed-...
...,...,...,...,...
679,Erioderma sorediatum,Endangered,open,ALA_DR654_679
680,Xanthoparmelia amphixantha,Endangered,open,ALA_DR654_680
681,Xanthoparmelia molliuscula,Endangered,open,ALA_DR654_681
682,Xanthoparmelia subloxodella,Endangered,open,ALA_DR654_682


In [16]:
parsednames['type'].unique()

array(['SCIENTIFIC', 'CULTIVAR', 'INFORMAL'], dtype=object)

In [24]:
## Testing to update duplicates
statelist

,scientificName,status,tas_geoprivacy,lsid
0,Lomatia tasmanica,Endangered,private,https://id.biodiversity.org.au/node/apni/2903191
2,Costora iena,Presumed Extinct,open,https://biodiversity.org.au/afd/taxa/7de808fb-...
3,Hadronyche pulvinator,Presumed Extinct,open,https://biodiversity.org.au/afd/taxa/2acd0df1-...
4,Hypolimnus pedderensis,Presumed Extinct,open,https://biodiversity.org.au/afd/taxa/7db09aa3-...
5,Thylacinus cynocephalus,Presumed Extinct,open,https://biodiversity.org.au/afd/taxa/97764bed-...
...,...,...,...,...
679,Erioderma sorediatum,Endangered,open,ALA_DR654_679
680,Xanthoparmelia amphixantha,Endangered,open,ALA_DR654_680
681,Xanthoparmelia molliuscula,Endangered,open,ALA_DR654_681
682,Xanthoparmelia subloxodella,Endangered,open,ALA_DR654_682


In [25]:
# check for duplicates with conflicting information
dupinformation = statelist.groupby('scientificName').filter(lambda x: len(x) > 1)#.sort('size',ascending=False)
dupinformation

,scientificName,status,tas_geoprivacy,lsid


### 4. Equivalent IUCN statuses

In [26]:
# iucn_statuses = {'Not Evaluated', 'Data Deficient', 'Least Concern', 'Near Threatened', 'Vulnerable', 'Endangered', 'Critically Endangered', 'Extinct in the Wild', 'Extinct'}
statelist.groupby(['status'])['status'].count()

status
Endangered          218
Presumed Extinct     28
Rare                308
Vulnerable          122
Name: status, dtype: int64

In [27]:
iucnStatusMappings = {
    'presumed extinct':'Extinct',
    'rare':'Vulnerable',
    'sensitive':'Vulnerable',
    'endangered': 'Endangered',
    'vulnerable': 'Vulnerable'
}

### 5. Determine best place ID to use

In [28]:
inatstatuses.groupby(['place_id','place_name','place_display_name'])['place_id'].count()
# looks like 6829

place_id  place_name          place_display_name    
6744      Australia           Australia                  2
6829      Tasmania            Tasmania, AU              77
9994      Northern Territory  Northern Territory, AU     6
Name: place_id, dtype: int64

## Merge iNaturalist statuses with State sensitive list on scientificName

1. Match - updates, even if the statuses are the same we'll update the links and values anyway
2. No match - statuses to be added (additions)
   1.1 No match and no taxnomy - search for synonyms
   1.2 No match
3. Merge the other direction to see if there are deletes?


In [40]:
# join to see which lists already have a status in inaturalist based on scientificName
mergedstatuses = statelist[['scientificName', 'status','tas_geoprivacy', 'lsid']].merge(inatstatuses[['status_id','scientificName','taxon_id','user_id','description','iucn','authority','status','geoprivacy','place_id','place_display_name']],how="left",left_on='scientificName',right_on='scientificName',suffixes=(None,'_inat')).sort_values(['scientificName'])
mergedstatuses


,scientificName,status,tas_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,authority,status_inat,geoprivacy,place_id,place_display_name
214,Acacia axillaris,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2911902,160726,775192,717122,,30,Environment Protection and Biodiversity Conser...,VU,obscured,6744,Australia
298,Acacia pataczekii,Rare,open,https://id.biodiversity.org.au/node/apni/2913609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
352,Acacia siculiformis,Rare,open,https://id.biodiversity.org.au/node/apni/2888960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
353,Acacia ulicifolia,Rare,open,https://id.biodiversity.org.au/node/apni/2907350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
354,Acacia uncifolia,Rare,open,https://id.biodiversity.org.au/node/apni/2916270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,Xerochrysum palustre,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2890165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,Zearaja maugeana,Endangered,open,https://biodiversity.org.au/afd/taxa/0d9d194f-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
519,Zieria littoralis,Rare,open,https://id.biodiversity.org.au/node/apni/2888662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
670,Zieria veronicea veronicea,Endangered,open,https://id.biodiversity.org.au/node/apni/2911634,161108,323791,717122,,40,Threatened Species Protection Act 1995,Endangered,obscured,6829,"Tasmania, AU"


In [41]:
# prepare the export fields, common to New template and Update template
# new statuses
# Taxon_Name,Status,Authority,IUCN_equivalent,Description,iNaturalist_Place_ID,url,Taxon_Geoprivacy,Username,taxon_id
# updates
# action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username
mergedstatuses['new_authority'] = "Threatened Species Protection Act 1995"
mergedstatuses['new_description'] = "Listed as Threatened - refer to https://www.threatenedspecieslink.tas.gov.au"
biesearchurl = "https://bie.ala.org.au/species/" # eg + "https://id.biodiversity.org.au/node/apni/2894366"
mergedstatuses['new_url'] =  biesearchurl + mergedstatuses['lsid']
mergedstatuses['new_geoprivacy'] = "obscured"
mergedstatuses['new_place_id'] = '6829'  # Tasmania
mergedstatuses['new_username'] = 'peggydnew'
mergedstatuses['new_iucn_equivalent'] = mergedstatuses['status'].str.lower().str.strip().map(iucnStatusMappings).fillna('Vulnerable') # map to dictionary
mergedstatuses['new_status'] = mergedstatuses['status'].fillna('Threatened')
mergedstatuses

,scientificName,status,tas_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,authority,...,place_id,place_display_name,new_authority,new_description,new_url,new_geoprivacy,new_place_id,new_username,new_iucn_equivalent,new_status
214,Acacia axillaris,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2911902,160726,775192,717122,,30,Environment Protection and Biodiversity Conser...,...,6744,Australia,Threatened Species Protection Act 1995,Listed as Threatened - refer to https://www.th...,https://bie.ala.org.au/species/https://id.biod...,obscured,6829,peggydnew,Vulnerable,Vulnerable
298,Acacia pataczekii,Rare,open,https://id.biodiversity.org.au/node/apni/2913609,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Threatened Species Protection Act 1995,Listed as Threatened - refer to https://www.th...,https://bie.ala.org.au/species/https://id.biod...,obscured,6829,peggydnew,Vulnerable,Rare
352,Acacia siculiformis,Rare,open,https://id.biodiversity.org.au/node/apni/2888960,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Threatened Species Protection Act 1995,Listed as Threatened - refer to https://www.th...,https://bie.ala.org.au/species/https://id.biod...,obscured,6829,peggydnew,Vulnerable,Rare
353,Acacia ulicifolia,Rare,open,https://id.biodiversity.org.au/node/apni/2907350,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Threatened Species Protection Act 1995,Listed as Threatened - refer to https://www.th...,https://bie.ala.org.au/species/https://id.biod...,obscured,6829,peggydnew,Vulnerable,Rare
354,Acacia uncifolia,Rare,open,https://id.biodiversity.org.au/node/apni/2916270,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Threatened Species Protection Act 1995,Listed as Threatened - refer to https://www.th...,https://bie.ala.org.au/species/https://id.biod...,obscured,6829,peggydnew,Vulnerable,Rare
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,Xerochrysum palustre,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2890165,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Threatened Species Protection Act 1995,Listed as Threatened - refer to https://www.th...,https://bie.ala.org.au/species/https://id.biod...,obscured,6829,peggydnew,Vulnerable,Vulnerable
171,Zearaja maugeana,Endangered,open,https://biodiversity.org.au/afd/taxa/0d9d194f-...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Threatened Species Protection Act 1995,Listed as Threatened - refer to https://www.th...,https://bie.ala.org.au/species/https://biodive...,obscured,6829,peggydnew,Endangered,Endangered
519,Zieria littoralis,Rare,open,https://id.biodiversity.org.au/node/apni/2888662,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Threatened Species Protection Act 1995,Listed as Threatened - refer to https://www.th...,https://bie.ala.org.au/species/https://id.biod...,obscured,6829,peggydnew,Vulnerable,Rare
670,Zieria veronicea veronicea,Endangered,open,https://id.biodiversity.org.au/node/apni/2911634,161108,323791,717122,,40,Threatened Species Protection Act 1995,...,6829,"Tasmania, AU",Threatened Species Protection Act 1995,Listed as Threatened - refer to https://www.th...,https://bie.ala.org.au/species/https://id.biod...,obscured,6829,peggydnew,Endangered,Endangered


## Updates

In [42]:
# updates
# action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
updates = pd.DataFrame(mergedstatuses[mergedstatuses['status_id'].notnull()])
updates.sort_values('scientificName')
updates['action'] = 'UPDATE'
#updates.loc[:,'action'] = 'UPDATE'
updates = updates[['action','scientificName','status_id','taxon_id','new_status','new_iucn_equivalent','new_authority','new_url','new_geoprivacy','new_place_id','new_username','new_description']]
updates.columns = updates.columns.str.replace("new_", "", regex=True)
updates = updates.rename(columns={'scientificName':'taxon_name',
                                  'status_id':'id'})
updates

,action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
214,UPDATE,Acacia axillaris,160726,775192,Vulnerable,Vulnerable,Threatened Species Protection Act 1995,https://bie.ala.org.au/species/https://id.biod...,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
598,UPDATE,Alternanthera denticulata,160727,369665,Endangered,Endangered,Threatened Species Protection Act 1995,https://bie.ala.org.au/species/https://id.biod...,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
599,UPDATE,Amphibromus macrorhinus,160728,1121410,Endangered,Endangered,Threatened Species Protection Act 1995,https://bie.ala.org.au/species/https://id.biod...,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
241,UPDATE,Anogramma leptophylla,162081,180646,Vulnerable,Vulnerable,Threatened Species Protection Act 1995,https://bie.ala.org.au/species/https://id.biod...,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
357,UPDATE,Aphelia gracilis,163877,1143399,Rare,Vulnerable,Threatened Species Protection Act 1995,https://bie.ala.org.au/species/https://id.biod...,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
...,...,...,...,...,...,...,...,...,...,...,...,...
663,UPDATE,Thelymitra malvina,161104,406524,Endangered,Endangered,Threatened Species Protection Act 1995,https://bie.ala.org.au/species/https://id.biod...,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
666,UPDATE,Veronica notabilis,161105,791936,Endangered,Endangered,Threatened Species Protection Act 1995,https://bie.ala.org.au/species/https://id.biod...,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
667,UPDATE,Viminaria juncea,161106,353911,Endangered,Endangered,Threatened Species Protection Act 1995,https://bie.ala.org.au/species/https://id.biod...,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
669,UPDATE,Wurmbea latifolia vanessae,161107,790432,Endangered,Endangered,Threatened Species Protection Act 1995,https://bie.ala.org.au/species/https://id.biod...,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...


## No status in iNaturalist via straight scientificName match
The ACT records that didn't match up to a status in iNaturalist

In [43]:
# to add: those that have no inaturalist status - 532!!
noinatstatus = mergedstatuses[mergedstatuses['status_id'].isnull()]
# try to match the taxon name to something in inaturalist
noinatstatus = noinatstatus.merge(inattaxa, how="left", left_on="scientificName",right_on="scientificName")
noinatstatus

,scientificName,status,tas_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,authority,...,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,taxonRank,references
0,Acacia pataczekii,Rare,open,https://id.biodiversity.org.au/node/apni/2913609,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,pataczekii,NaN,2022-04-06T23:48:02Z,species,http://www.catalogueoflife.org/annual-checklis...
1,Acacia siculiformis,Rare,open,https://id.biodiversity.org.au/node/apni/2888960,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,siculiformis,NaN,2022-04-07T00:40:40Z,species,http://www.catalogueoflife.org/annual-checklis...
2,Acacia ulicifolia,Rare,open,https://id.biodiversity.org.au/node/apni/2907350,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,ulicifolia,NaN,2022-06-18T06:33:18Z,species,http://www.catalogueoflife.org/annual-checklis...
3,Acacia uncifolia,Rare,open,https://id.biodiversity.org.au/node/apni/2916270,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,uncifolia,NaN,2022-06-10T10:52:23Z,species,http://eol.org/pages/26965172
4,Acanthiza pusilla archibaldi,Endangered,open,ALA_DR654_159,NaN,NaN,NaN,NaN,NaN,NaN,...,Chordata,Aves,Passeriformes,Acanthizidae,Acanthiza,pusilla,archibaldi,2018-12-19T06:16:34Z,subspecies,http://www.birds.cornell.edu/clementschecklist...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,Xerochrysum bicolor,Rare,open,https://id.biodiversity.org.au/node/apni/2902500,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Asterales,Asteraceae,Xerochrysum,bicolor,NaN,2020-02-19T08:20:14Z,species,http://www.catalogueoflife.org/annual-checklis...
599,Xerochrysum palustre,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2890165,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Asterales,Asteraceae,Xerochrysum,palustre,NaN,2022-06-08T17:48:57Z,species,http://eol.org/pages/5127428
600,Zearaja maugeana,Endangered,open,https://biodiversity.org.au/afd/taxa/0d9d194f-...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
601,Zieria littoralis,Rare,open,https://id.biodiversity.org.au/node/apni/2888662,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Sapindales,Rutaceae,Zieria,littoralis,NaN,2022-06-08T17:54:24Z,species,https://eol.org/pages/52379143


In [44]:
additions = pd.DataFrame(noinatstatus[noinatstatus['id'].notna()])
additions

,scientificName,status,tas_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,authority,...,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,taxonRank,references
0,Acacia pataczekii,Rare,open,https://id.biodiversity.org.au/node/apni/2913609,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,pataczekii,NaN,2022-04-06T23:48:02Z,species,http://www.catalogueoflife.org/annual-checklis...
1,Acacia siculiformis,Rare,open,https://id.biodiversity.org.au/node/apni/2888960,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,siculiformis,NaN,2022-04-07T00:40:40Z,species,http://www.catalogueoflife.org/annual-checklis...
2,Acacia ulicifolia,Rare,open,https://id.biodiversity.org.au/node/apni/2907350,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,ulicifolia,NaN,2022-06-18T06:33:18Z,species,http://www.catalogueoflife.org/annual-checklis...
3,Acacia uncifolia,Rare,open,https://id.biodiversity.org.au/node/apni/2916270,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,uncifolia,NaN,2022-06-10T10:52:23Z,species,http://eol.org/pages/26965172
4,Acanthiza pusilla archibaldi,Endangered,open,ALA_DR654_159,NaN,NaN,NaN,NaN,NaN,NaN,...,Chordata,Aves,Passeriformes,Acanthizidae,Acanthiza,pusilla,archibaldi,2018-12-19T06:16:34Z,subspecies,http://www.birds.cornell.edu/clementschecklist...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,Xanthorrhoea arenaria,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2918552,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Liliopsida,Asparagales,Asphodelaceae,Xanthorrhoea,arenaria,NaN,2019-02-15T07:38:04Z,species,http://www.ubio.org/browser/details.php?nameba...
597,Xanthorrhoea bracteata,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2911139,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Liliopsida,Asparagales,Asphodelaceae,Xanthorrhoea,bracteata,NaN,2022-08-25T07:03:45Z,species,http://powo.science.kew.org/
598,Xerochrysum bicolor,Rare,open,https://id.biodiversity.org.au/node/apni/2902500,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Asterales,Asteraceae,Xerochrysum,bicolor,NaN,2020-02-19T08:20:14Z,species,http://www.catalogueoflife.org/annual-checklis...
599,Xerochrysum palustre,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2890165,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Asterales,Asteraceae,Xerochrysum,palustre,NaN,2022-06-08T17:48:57Z,species,http://eol.org/pages/5127428


In [45]:
# there's no status but there is a matching inat taxon (id is the taxon id)
additions = pd.DataFrame(noinatstatus[noinatstatus['id'].notna()])
additions.sort_values(['scientificName'])
additions['action'] = 'ADD'
additions = additions[['action','scientificName','status_id','id','new_status','new_iucn_equivalent','new_authority','new_url','new_geoprivacy','new_place_id','new_username','new_description']]
additions.columns = additions.columns.str.replace("new_", "", regex=True)
additions = additions.rename(columns={'scientificName':'taxon_name',
                                      'id':'taxon_id',
                                  'status_id':'id'})
additions

,action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
0,ADD,Acacia pataczekii,NaN,1254534,Rare,Vulnerable,Threatened Species Protection Act 1995,https://bie.ala.org.au/species/https://id.biod...,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
1,ADD,Acacia siculiformis,NaN,567724,Rare,Vulnerable,Threatened Species Protection Act 1995,https://bie.ala.org.au/species/https://id.biod...,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
2,ADD,Acacia ulicifolia,NaN,139880,Rare,Vulnerable,Threatened Species Protection Act 1995,https://bie.ala.org.au/species/https://id.biod...,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
3,ADD,Acacia uncifolia,NaN,560392,Rare,Vulnerable,Threatened Species Protection Act 1995,https://bie.ala.org.au/species/https://id.biod...,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
4,ADD,Acanthiza pusilla archibaldi,NaN,629326,Endangered,Endangered,Threatened Species Protection Act 1995,https://bie.ala.org.au/species/ALA_DR654_159,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
...,...,...,...,...,...,...,...,...,...,...,...,...
596,ADD,Xanthorrhoea arenaria,NaN,125126,Vulnerable,Vulnerable,Threatened Species Protection Act 1995,https://bie.ala.org.au/species/https://id.biod...,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
597,ADD,Xanthorrhoea bracteata,NaN,895045,Vulnerable,Vulnerable,Threatened Species Protection Act 1995,https://bie.ala.org.au/species/https://id.biod...,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
598,ADD,Xerochrysum bicolor,NaN,147469,Rare,Vulnerable,Threatened Species Protection Act 1995,https://bie.ala.org.au/species/https://id.biod...,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
599,ADD,Xerochrysum palustre,NaN,634811,Vulnerable,Vulnerable,Threatened Species Protection Act 1995,https://bie.ala.org.au/species/https://id.biod...,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...


In [46]:
# write these to the file
pd.concat([updates,additions]).to_csv(sourcedir + "tas.csv", index=False)

In [47]:
# what didnt match to a taxon?
unknownToInat = noinatstatus[noinatstatus['id'].isna()]
unknownToInat

,scientificName,status,tas_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,authority,...,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,taxonRank,references
8,Agrostis australiensis,Rare,open,https://id.biodiversity.org.au/node/apni/2907094,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Agrostis diemenica,Rare,open,https://id.biodiversity.org.au/node/apni/2905826,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,Austromelanelixia piliferella,Vulnerable,open,ALA_DR654_288,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,Austroparmelina pallida,Endangered,open,ALA_DR654_600,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,Austroparmelina whinrayi,Rare,open,ALA_DR654_351,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,Xanthoparmelia microphyllizans,Rare,open,ALA_DR654_530,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
593,Xanthoparmelia vicaria,Rare,open,ALA_DR654_353,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
594,Xanthoparmelia vicariella,Rare,open,ALA_DR654_354,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
600,Zearaja maugeana,Endangered,open,https://biodiversity.org.au/afd/taxa/0d9d194f-...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### are there any that need to be removed?
TAS sensitive list count: 1
TAS conservation list count: 683
TAS inat statuses count: 0

updates to inat status: 67
additional inat status: 0
TAS statuses we can't find a taxon match for in iNaturalist: 617
total: (explainable via the various genus/section entries that we matched to in the taxonomy)

inat statuses left over:  that may need checking

In [48]:
# inat statuses that aren't in added or updated
notaddedupdated = inatstatuses[~inatstatuses['taxon_id'].isin(updates['taxon_id'])]
notaddedupdated

,status_id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,...,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
3353,267438,1413424,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Pneumatopt...,,obscured,...,Pakau,pennigera,,2022-09-25T23:02:37Z,Pakau pennigera,species,https://shop.brit.org/A-Generic-Classification...,,,
1251,152435,19250,708886,9994,16652,NT NRETAS,endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Polytelis,alexandrae,,2019-08-27T01:09:01Z,Polytelis alexandrae,species,http://www.birdlife.org/datazone/speciesfactsh...,,,
904,160736,353835,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Blechnum-n...,,obscured,...,Doodia,aspera,,2021-02-22T18:52:06Z,Doodia aspera,species,http://www.ubio.org/browser/details.php?nameba...,,,
1249,152433,38633,708886,9994,16652,NT NRETAS,endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Bellatorias,obiri,,2019-04-30T15:19:21Z,Bellatorias obiri,species,http://reptile-database.reptarium.cz/search.ph...,,,
932,160737,401378,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Blechnum%2...,,obscured,...,Doodia,caudata,,2021-02-15T17:49:20Z,Doodia caudata,species,,,,
959,160741,405825,717122,6829,,Threatened Species Protection Act 1995,Presumed extinct,https://dpipwe.tas.gov.au/Documents/Botrychium...,,obscured,...,,,,,Sceptridium australe,,,parsley fern,False,[400050]
1252,152436,40743,708886,9994,16652,NT NRETAS,endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,,,,,Hipposideros stenotis,,,Narrow-eared Roundleaf Bat,False,[1431118]
1248,152432,41326,708886,9994,16652,NT NRETAS,endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Macroderma,gigas,,2019-08-27T01:58:05Z,Macroderma gigas,species,http://www.catalogueoflife.org/annual-checklis...,,,
2588,160758,446738,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Corunastyl...,,obscured,...,Genoplesium,morrisii,,2020-08-17T06:18:44Z,Genoplesium morrisii,species,http://www.catalogueoflife.org/annual-checklis...,,,
1247,152431,73180,708886,9994,16652,NT NRETAS,endangered,https://lists.ala.org.au/speciesListItem/list/...,,private,...,Pezoporus,occidentalis,,2019-08-27T01:09:27Z,Pezoporus occidentalis,species,http://www.birdlife.org/datazone/speciesfactsh...,,,


In [49]:
# Stats
numsensitive = len(sensitivelist.index)
numconservation = len(conservationlist.index)
numupdates  = len(updates.index)
numadditions  = len(additions.index)
numnoinatstatus = len(noinatstatus.index)
numunknownToInat = len(unknownToInat.index)
numnotaddedupdated = len(notaddedupdated.index)
numnoncomply = len(noncomply.index)
numcomply = len(statelist.index)
numdupinfo = len(dupinformation.index)
d = {'Sensitive': [numsensitive],
    'Conservation': [numconservation],
    'Statelist merge': [numfullstatelist],
    'Species iNat Comply' : [numcomply],
    'Species iNat non-Comply': [numnoncomply],
    'Duplicate Information': [numdupinfo],
    'Updates': [numupdates],
    'Additions': [numadditions],
    'Not added updated': [numnotaddedupdated],
    'No Inat Status': [numnoinatstatus],
    'Unknown to Inat': [numunknownToInat]}

statsdf = pd.DataFrame(data=d)
statsdf

,Sensitive,Conservation,Statelist merge,Species iNat Comply,Species iNat non-Comply,Duplicate Information,Updates,Additions,Not added updated,No Inat Status,Unknown to Inat
0,1,683,684,676,7,0,73,500,12,603,103
